# Tutorial 2: Catenary Analysis Deep Dive

**Duration:** ~30 minutes  
**Level:** Intermediate  
**Prerequisites:** Tutorial 1

## Learning Objectives

1. Compare simplified vs boundary value problem (BVP) solvers
2. Perform parameter sensitivity analysis
3. Analyze multi-segment lazy-wave risers
4. Apply to real FPSO case study
5. Optimize mooring design

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent.parent / 'src'))

from marine_engineering.catenary.solver import CatenarySolver, CatenaryInput
from marine_engineering.catenary.simplified import SimplifiedCatenary
from marine_engineering.catenary.lazy_wave import LazyWaveRiser

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Solver Comparison: Simplified vs BVP

Let's compare the simplified catenary solver with the full BVP solver.

In [ ]:
# Test case parameters
test_cases = [
    {'name': 'Shallow Catenary', 'L': 1000, 'X': 950, 'Y': 50, 'w': 800, 'EA': 1e9},
    {'name': 'Deep Catenary', 'L': 2000, 'X': 1500, 'Y': 500, 'w': 1200, 'EA': 1.5e9},
    {'name': 'Steep Riser', 'L': 500, 'X': 200, 'Y': 400, 'w': 1500, 'EA': 800e6},
]

comparison_results = []

for case in test_cases:
    # Simplified solver
    simple = SimplifiedCatenary(
        length=case['L'],
        horizontal_span=case['X'],
        weight_per_length=case['w'],
        ea_stiffness=case['EA']
    )
    H_simple, _ = simple.solve()
    
    # BVP solver
    params = CatenaryInput(
        length=case['L'],
        horizontal_span=case['X'],
        vertical_span=case['Y'],
        weight_per_length=case['w'],
        ea_stiffness=case['EA']
    )
    solver = CatenarySolver()
    results_bvp = solver.solve(params)
    H_bvp = results_bvp.horizontal_tension
    
    # Compare
    error = abs(H_simple - H_bvp) / H_bvp * 100
    
    comparison_results.append({
        'Case': case['name'],
        'H_Simplified [kN]': H_simple/1000,
        'H_BVP [kN]': H_bvp/1000,
        'Error [%]': error,
        'L/X Ratio': case['L']/case['X']
    })

df_comparison = pd.DataFrame(comparison_results)
print("\n" + "="*70)
print("SOLVER COMPARISON RESULTS")
print("="*70)
print(df_comparison.to_string(index=False))
print("="*70)
print(f"\n✓ Simplified solver is accurate for shallow catenaries (L/X < 1.1)")
print(f"✓ Use BVP solver for steep risers or large vertical spans")

## 2. Parameter Sensitivity Analysis

Analyze how mooring tensions respond to parameter variations.

In [ ]:
# Base case
base_params = {
    'length': 1500,
    'horizontal_span': 1200,
    'vertical_span': 100,
    'weight_per_length': 1000,
    'ea_stiffness': 1e9
}

# Sensitivity analysis parameters
param_ranges = {
    'length': np.linspace(1200, 1800, 20),
    'weight_per_length': np.linspace(600, 1400, 20),
    'ea_stiffness': np.linspace(0.5e9, 2e9, 20),
    'horizontal_span': np.linspace(1000, 1400, 20)
}

# Perform sensitivity analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

solver = CatenarySolver()

for idx, (param_name, param_values) in enumerate(param_ranges.items()):
    tensions = []
    elongations = []
    
    for val in param_values:
        test_params = base_params.copy()
        test_params[param_name] = val
        
        params = CatenaryInput(**test_params)
        try:
            results = solver.solve(params)
            tensions.append(results.total_tension_fairlead / 1000)  # kN
            elongations.append(results.elongation)
        except:
            tensions.append(np.nan)
            elongations.append(np.nan)
    
    ax = axes[idx]
    ax2 = ax.twinx()
    
    l1 = ax.plot(param_values, tensions, 'b-', linewidth=2, label='Tension')
    l2 = ax2.plot(param_values, elongations, 'r--', linewidth=2, label='Elongation')
    
    ax.set_xlabel(param_name.replace('_', ' ').title(), fontsize=11)
    ax.set_ylabel('Tension [kN]', fontsize=11, color='b')
    ax2.set_ylabel('Elongation [m]', fontsize=11, color='r')
    ax.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    ax.set_title(f'Sensitivity to {param_name.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Combined legend
    lns = l1 + l2
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc='best', fontsize=9)

plt.suptitle('Mooring Line Parameter Sensitivity Analysis', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../output/tutorial2_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Lazy-Wave Riser Analysis

Multi-segment lazy-wave configuration for SCR applications.

In [ ]:
# Define lazy-wave riser segments
lazy_wave = LazyWaveRiser()

# Segment definitions
segments = [
    {'type': 'chain', 'length': 300, 'weight': 1500, 'EA': 1.2e9, 'label': 'Bottom Chain'},
    {'type': 'buoyancy', 'length': 200, 'weight': -800, 'EA': 800e6, 'label': 'Buoyancy Section'},
    {'type': 'pipe', 'length': 500, 'weight': 1200, 'EA': 1e9, 'label': 'Steel Pipe'},
    {'type': 'chain', 'length': 200, 'weight': 1500, 'EA': 1.2e9, 'label': 'Top Chain'}
]

# Calculate lazy-wave shape
x_total = []
y_total = []
tension_total = []
x_current = 0
y_current = 0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

colors = ['brown', 'orange', 'blue', 'brown']

for idx, seg in enumerate(segments):
    # Simplified segment calculation
    params = CatenaryInput(
        length=seg['length'],
        horizontal_span=seg['length'] * 0.8,
        vertical_span=50 if idx < 2 else -30,
        weight_per_length=seg['weight'],
        ea_stiffness=seg['EA']
    )
    
    solver = CatenarySolver()
    results = solver.solve(params)
    
    # Offset coordinates
    x_seg = results.shape_x + x_current
    y_seg = results.shape_y + y_current
    
    x_total.extend(x_seg)
    y_total.extend(y_seg)
    tension_total.extend(results.tension_distribution)
    
    # Plot segment
    ax1.plot(x_seg, y_seg, linewidth=3, color=colors[idx], label=seg['label'])
    
    # Update position
    x_current = x_seg[-1]
    y_current = y_seg[-1]

# Configuration plot
ax1.axhline(y=0, color='brown', linestyle='--', linewidth=1.5, label='Seabed', alpha=0.6)
ax1.plot(x_total[0], y_total[0], 'go', markersize=12, label='Anchor', zorder=5)
ax1.plot(x_total[-1], y_total[-1], 'ro', markersize=12, label='Fairlead', zorder=5)
ax1.fill_between([min(x_total), max(x_total)], -50, 0, color='brown', alpha=0.2)
ax1.set_xlabel('Horizontal Distance [m]', fontsize=12)
ax1.set_ylabel('Vertical Distance [m]', fontsize=12)
ax1.set_title('Lazy-Wave Riser Configuration', fontsize=13, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-60, max(y_total) + 20)

# Tension distribution
ax2.plot(x_total, np.array(tension_total)/1000, 'purple', linewidth=2.5)
ax2.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('Horizontal Distance [m]', fontsize=12)
ax2.set_ylabel('Tension [kN]', fontsize=12)
ax2.set_title('Tension Distribution Along Riser', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add segment markers
segment_positions = [0]
cumulative_x = 0
for seg in segments:
    cumulative_x += seg['length'] * 0.8
    segment_positions.append(cumulative_x)
    ax2.axvline(x=cumulative_x, color='gray', linestyle=':', alpha=0.5)

plt.suptitle('Lazy-Wave Riser Multi-Segment Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../output/tutorial2_lazy_wave.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Lazy-wave configuration provides:")
print("  - Reduced seabed interaction")
print("  - Lower fatigue at touchdown zone")
print("  - Decoupled vessel motions")

## 4. Real FPSO Case Study

Design mooring system for 200,000 DWT FPSO in 1500m water depth.

In [ ]:
# FPSO mooring design parameters
fpso_data = {
    'name': 'FPSO Atlantis',
    'displacement': 200000,  # tonnes
    'water_depth': 1500,  # m
    'num_lines': 12,
    'line_length': 2500,  # m
    'horizontal_offset': 1800,  # m (10% water depth)
    'vertical_offset': 100,  # m (fairlead above anchor)
    'chain_diameter': 127,  # mm (R4 studlink)
    'chain_weight': 3490,  # N/m in water
    'chain_EA': 1.47e9,  # N
}

# Analyze single mooring line
params = CatenaryInput(
    length=fpso_data['line_length'],
    horizontal_span=fpso_data['horizontal_offset'],
    vertical_span=fpso_data['vertical_offset'],
    weight_per_length=fpso_data['chain_weight'],
    ea_stiffness=fpso_data['chain_EA'],
    water_depth=fpso_data['water_depth']
)

solver = CatenarySolver()
results = solver.solve(params)

# Calculate system capacity
total_capacity = results.total_tension_fairlead * fpso_data['num_lines']
design_safety_factor = 1.67  # API RP 2SK

print("\n" + "="*70)
print(f"FPSO MOORING SYSTEM ANALYSIS: {fpso_data['name']}")
print("="*70)
print(f"\nVessel Details:")
print(f"  Displacement:           {fpso_data['displacement']:,} tonnes")
print(f"  Water Depth:            {fpso_data['water_depth']:,} m")
print(f"  Number of Lines:        {fpso_data['num_lines']}")
print(f"\nMooring Line (per line):")
print(f"  Chain Diameter:         {fpso_data['chain_diameter']} mm R4 Studlink")
print(f"  Line Length:            {fpso_data['line_length']:,} m")
print(f"  Horizontal Offset:      {fpso_data['horizontal_offset']:,} m")
print(f"\nCalculated Tensions:")
print(f"  Horizontal Tension:     {results.horizontal_tension/1e6:.2f} MN")
print(f"  Fairlead Tension:       {results.total_tension_fairlead/1e6:.2f} MN")
print(f"  Anchor Tension:         {results.total_tension_anchor/1e6:.2f} MN")
print(f"  Line Elongation:        {results.elongation:.1f} m")
print(f"\nSystem Performance:")
print(f"  Total System Capacity:  {total_capacity/1e6:.1f} MN ({fpso_data['num_lines']} lines)")
print(f"  Per-line Restoring:     {results.horizontal_tension/1e6:.2f} MN/line")
print(f"  Touchdown Distance:     {results.touchdown_distance:.1f} m" if results.touchdown_distance else "  No touchdown calculated")
print(f"\nSafety Check (API RP 2SK):")
print(f"  Design Safety Factor:   {design_safety_factor:.2f}")
print(f"  MBL Required (127mm):   {results.total_tension_fairlead * design_safety_factor / 1e6:.2f} MN")
print(f"  R4 Studlink 127mm MBL:  ~15.4 MN ✓")
print("="*70)

## 5. Design Optimization

Find optimal line configuration minimizing cost while meeting requirements.

In [ ]:
from scipy.optimize import minimize

# Optimization objective: minimize line length (cost) while meeting tension limit
def optimization_objective(x, target_H, X, Y, w, EA):
    """Objective: minimize line length while achieving target horizontal tension"""
    L = x[0]
    
    if L < np.sqrt(X**2 + Y**2):  # Infeasible
        return 1e10
    
    try:
        params = CatenaryInput(
            length=L,
            horizontal_span=X,
            vertical_span=Y,
            weight_per_length=w,
            ea_stiffness=EA
        )
        solver = CatenarySolver()
        results = solver.solve(params)
        
        # Penalty for not meeting target tension
        H_error = abs(results.horizontal_tension - target_H) / target_H
        
        return L * (1 + 100 * H_error)  # Length + penalty
    except:
        return 1e10

# Optimization parameters
target_horizontal_tension = 3.5e6  # 3.5 MN target
X = 1500  # Fixed horizontal span
Y = 80    # Fixed vertical span
w = 2800  # Chain weight N/m
EA = 1.2e9  # Chain stiffness

# Initial guess
L_initial = 2000

# Run optimization
result = minimize(
    optimization_objective,
    [L_initial],
    args=(target_horizontal_tension, X, Y, w, EA),
    method='Nelder-Mead',
    options={'maxiter': 100}
)

L_optimal = result.x[0]

# Verify optimized design
params_opt = CatenaryInput(
    length=L_optimal,
    horizontal_span=X,
    vertical_span=Y,
    weight_per_length=w,
    ea_stiffness=EA
)
results_opt = solver.solve(params_opt)

print("\n" + "="*70)
print("MOORING LINE OPTIMIZATION RESULTS")
print("="*70)
print(f"Target Horizontal Tension:  {target_horizontal_tension/1e6:.2f} MN")
print(f"Achieved Horizontal Tension: {results_opt.horizontal_tension/1e6:.2f} MN")
print(f"\nOptimal Line Length:        {L_optimal:.1f} m")
print(f"Initial Guess:              {L_initial} m")
print(f"Length Saved:               {L_initial - L_optimal:.1f} m ({(L_initial-L_optimal)/L_initial*100:.1f}%)")
print(f"\nCost Implications:")
print(f"  Chain cost: ~$1000/m")
print(f"  Savings per line:           ${(L_initial - L_optimal) * 1000:,.0f}")
print(f"  12-line system savings:     ${(L_initial - L_optimal) * 1000 * 12:,.0f}")
print("="*70)

## Summary

### Key Takeaways

✅ **Solver Selection**
- Simplified solver: Fast, accurate for L/X < 1.1
- BVP solver: Required for steep risers and large vertical spans

✅ **Parameter Sensitivity**
- Line length: Most sensitive parameter
- Weight: Direct impact on tensions
- EA: Affects elongation more than tension

✅ **Multi-Segment Analysis**
- Lazy-wave reduces fatigue
- Buoyancy sections decouple vessel motions
- Complex configurations need careful design

✅ **Real-World Application**
- FPSO systems: 12-16 lines typical
- Safety factors: 1.67-2.0 per API RP 2SK
- Optimization can save significant costs

### Next: Tutorial 3 - Wave Spectra Mastery